#### According to research, the Google vision API is not robust to noise

#### pip install numpy opencv-python

In [1]:
import io
import os
import pandas as pd 
import time
from google.cloud import vision_v1 as vision
from google.oauth2 import service_account
import cv2  # Import OpenCV

In [5]:
def detect_landmarks_local(image_path):
    """Detects landmarks in a local file, resizing if the image is too large."""
    client = vision.ImageAnnotatorClient()

    try:
        # Read the image using OpenCV
        image = cv2.imread(image_path)

        # Check if the image is too large, resize if yes. (Processing Error Bad Image, that's why Decompress)
        if os.path.getsize(image_path) > 6e6:  # size in bytes, 10 MB limit
            scale_ratio = (6e6 / os.path.getsize(image_path))**0.5  # Scales down the image
            image = cv2.resize(image, (int(image.shape[1] * scale_ratio), int(image.shape[0] * scale_ratio)))

        # Convert the image back to bytes for API call
        _, buffer = cv2.imencode('.jpg', image)
        content = buffer.tobytes()

        # Create a Vision API image object
        image = vision.Image(content=content)

        start_time = time.time()
        response = client.landmark_detection(image=image)
        end_time = time.time()

        response_time = end_time - start_time

        if response.error.message:
            raise Exception(
                "{}\nFor more info on error messages, check: "
                "https://cloud.google.com/apis/design/errors".format(response.error.message)
            )

        landmarks = response.landmark_annotations
        if landmarks:
            top_landmark = max(landmarks, key=lambda x: x.score)
            description = top_landmark.description
            score = top_landmark.score
        else:
            description = None
            score = 0.0

    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return image_path, None, 0.0, None

    return image_path, description, score, response_time

In [3]:
def process_folder(folder_path):
    results = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            file_path = os.path.join(folder_path, filename)
            result = detect_landmarks_local(file_path)
            results.append(result)
    return results

In [7]:
folder_path = 'output_folder'
results = process_folder(folder_path)

Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file


Error processing output_folder/ZFOHXKIV6XQUCOGLRGN4XQDLFMDG5N2RWDDLMAOPKI5QNALNHVDA.jpg: 503 502:Bad Gateway


Premature end of JPEG file


In [8]:
df = pd.DataFrame(results, columns=['image_id', 'google_vision_sp_detection', 'google_vision_sp_score', 'google_vision_sp_response Time'])
print(df)

                                              image_id  \
0    output_folder/AQ7Q666WK2OXZKIQBQVWPRC5D2M6XFBE...   
1    output_folder/TKUAUGKMUUXUYKFE53MLJ64UC2LBDZK2...   
2    output_folder/5FF4Y4LHR5EYDJ7K25U5HUMKWANKYONM...   
3    output_folder/OQNB4FXPW3SUEF2CRCJNENGESSWUNL5X...   
4    output_folder/7LZ6QBXCSKYIUJUSJK2OO3VCHQ5CTWVZ...   
..                                                 ...   
266  output_folder/WTTKN72SLSR7MMY5TVF6ITSI2PT3WZTY...   
267  output_folder/KQAGOP6B4V4GKGXQNDDLQDRY2PLQ35LG...   
268  output_folder/AKM4JAMFM7CCHTFK72ACCQMHZ5LF6POZ...   
269  output_folder/DPCW35BXU2KSMYUIGNDWCMADZZEAQMP6...   
270  output_folder/LIPE3D2KSBIVOBBIEQL5HQMM46HIBOUA...   

    google_vision_sp_detection  google_vision_sp_score  \
0                         None                0.000000   
1            The Walmer Castle                0.475537   
2                         None                0.000000   
3                         None                0.000000   
4            

In [10]:
df['image_id'] = df['image_id'].str.split('/').str[-1]

In [13]:
valid_classes = ['Tower Bridge', 'The British Museum', 'Big Ben', 'Lastminute.com London Eye', 'Tower Of London', 'Trafalgar Square', 'Westminster Bridge', '30 St Mary Axe (The Gherkin)', 'Palace Of Westminster', 'The Shard', 'Wembley Stadium','None']

# Update 'google_vision_sp_detection' and 'google_vision_sp_score' based on the condition
df.loc[~df['google_vision_sp_detection'].isin(valid_classes), 'google_vision_sp_detection'] = None
df.loc[~df['google_vision_sp_detection'].isin(valid_classes), 'google_vision_sp_score'] = 0.0


In [16]:
df.to_csv('google_vision_sp_detection.csv', index=False)
df


,image_id,google_vision_sp_detection,google_vision_sp_score,google_vision_sp_response Time
5,VJAEXBJJFXCXG5QOFBJHHXETMXGPBW46WMQFMCAHG774TR...,Lastminute.com London Eye,0.255765,0.339404
8,DBY224LJBG3JDE7JLZRXKKZEPD632LR23WRZ7TBBP75KZ2...,Tower Bridge,0.345929,0.538117
17,BSLUHXKIS375LYW2GKV4ZGZ4PCW5KQYT5J36XAIARPV2ZH...,Tower Bridge,0.533501,0.206307
18,72A7RGYIHGG67WDKRVE5QFGDGSPOGTMK7Y54LL3U35364Q...,The British Museum,0.258308,0.376178
24,BZ5DDCQ7YTWQIMJCCOE42QI4A66KDHU6P37UHV5NPD6AQ2...,Tower Of London,0.434017,0.376596
...,...,...,...,...
244,XFAS4LJMQUNYWTZ5DUEGX6C5ZYAQE5DHTHBNPSTPIMOMZ5...,Tower Bridge,0.327560,0.391788
245,AIB4QMIJVIKNTFF6J3NBVSJDUH6O6XF4OFJJ3CBIBLZEK4...,Tower Bridge,0.512237,0.201002
247,LWNHMH7KUJRGPXAWRUBWEY4ERTTFFA3JPZWVBQQY6SHOYA...,Tower Of London,0.406702,0.386406
256,W7SFDA5HIZJIWIOXGJURQAIGPDQKXBYO45XCC4XCIKRFN4...,Tower Bridge,0.399474,0.340927


,Unnamed: 0,dt,accommodation_id,image_id,tag_name,probability,tag_id,image,name,latitude,...,description,score,response_time,label_name,bbox_x,bbox_y,bbox_width,bbox_height,image_width,image_height
0,0,2023-10-28,47101,326938012,Building,0.999986,2.0,https://imgcy.trivago.com/partnerimages/52/42/...,Park Plaza London Riverbank,51.49149,...,Big Ben,0.551586,0.221124,Big Ben,2250.0,580.0,349.0,1057.0,3840.0,2160.0
1,1,2023-10-28,29265342,901351302,NaN,NaN,NaN,https://imgcy.trivago.com/partnerimages/16/85/...,Homely 3 Bedroom House With A Garden,51.49630,...,Big Ben,0.961087,0.235555,Big Ben,2268.0,391.0,379.0,1166.0,2880.0,1920.0
2,2,2023-10-28,14940268,230453664,Swimming Pool,0.999996,723.0,https://imgcy.trivago.com/partnerimages/44/98/...,Central London 1 Bedroom Riverside Luxury Apar...,51.49367,...,Big Ben,0.840975,0.160658,Big Ben,543.0,47.0,180.0,423.0,1024.0,683.0
3,3,2023-10-28,13522206,848646685,NaN,NaN,NaN,https://imgcy.trivago.com/partnerimages/15/82/...,A Brand New 747 Square Foot 2 Bedroom Luxury A...,51.50722,...,Big Ben,0.329953,0.276659,Big Ben,1830.0,1439.0,243.0,611.0,3840.0,2563.0
4,4,2023-10-28,34312250,941094282,NaN,NaN,NaN,https://imgcy.trivago.com/partnerimages/17/64/...,Sleek & Cosy 1bd Flat - Clapham Common,51.50722,...,Big Ben,0.882675,0.194696,Big Ben,439.0,557.0,197.0,537.0,1280.0,1920.0


,Image_name,Description,Score,Response Time
0,857267694,None,0.000000,0.491437
1,803428234,None,0.000000,0.119054
2,393050685,Westminster Bridge,0.276943,0.116712
3,803428163,None,0.000000,0.122654
4,939862109,None,0.000000,0.565565


True Positives: 7
False Positives: 0
